In [1]:
# Installing Libraries

%pip install selenium
%pip install time

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


In [2]:
# Enter your username i.e. C:\Users\stude\AppData\ In this case it would be 'stude'
username = "stude"

In [3]:
# Set it to false if you don't want the operations being done in Chrome to be visible
view_operation = True

In [4]:
# Enter your queries in the format specified below:
# Each query must be a different element of the list
# Ensure to use \n if you want to type on multiple lines

import json
# Read File

file_name = "MaSaC_val_erc.json"
with open(file_name, 'r') as file:
        dataset = json.load(file)
# Number of episodes in dataset

N = len(dataset)
print(N)
# Reading the data

episodes_list = []
speakers_list = []
utterances_list = []
emotions_list = []

for i in range(len(dataset)):
        episodes_list.append(dataset[i]['episode'])
        speakers_list.append(dataset[i]['speakers'])
        utterances_list.append(dataset[i]['utterances'])
        emotions_list.append(dataset[i]['emotions'])  
    
base_prompt = \
'''Perform ERC given sentences
The emotion labels MUST be the most accurate in this world.
It is fine if you take time but labels must be correct.
Assign an emotion label out of labels = ["neutral", "anger", "surprise", "fear", "joy", "contempt", "sadness", "disgust"]
The output should not contain anything except the list of emotion labels
The output should not contain input sentences
The output should not contain any explanation
The output should NOT NOT NOT NOT NOT contain any additional text like text from you like "Here is the list of emotion labels for the provided sentences:"
Only a python  list of emotions
OUTPUT SAMPLE:
["fear","neutral","sadness","neutral","fear"]

Now your INPUT is:

'''
# Reading the data
prompts = []
for i in range(len(dataset)):
        sp = "Speakers: " + str(speakers_list[i]) + "\n\n"
        utt  = "Utterances: " + str(utterances_list[i])
        prompts.append(base_prompt + sp + utt)
    
# Reading the data
prompts = []
for i in range(len(dataset)):
        sp = "Speakers: " + str(speakers_list[i]) + "\n\n"
        utt  = "Utterances: " + str(utterances_list[i])
        prompts.append(base_prompt + sp + utt)

queries = prompts

46


In [5]:
# Setting Variables

chrome_path = 'C:\\Users\\' + username + '\\AppData\\Local\\Google\\Chrome\\User Data'

In [6]:
# Importing Libraries

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

In [7]:
# Opening Chrome with your profile

my_options = webdriver.ChromeOptions() 
userdatadir = chrome_path
if view_operation==False:
    my_options.add_experimental_option("excludeSwitches", ['enable-automation'])
my_options.add_argument(f"--user-data-dir={userdatadir}")
driver = webdriver.Chrome(options=my_options)

In [8]:
# Opening Website

driver.get('https://chat.openai.com/')

In [9]:
# Waiting Functions Definations
            
# Waits for response to be typed
def wait_response(driver):
    flag = False
    
    while flag==False:
        time.sleep(0.1)
        try:
            text = driver.find_element(by=By.XPATH, value="/html/body/div[1]/div[1]/div[2]/main/div[1]/div[2]/form/div/div[1]/div/div[2]/div/button/div").text
        except:
            flag = False
        else:
            flag =  text=='Regenerate'

In [10]:
# Typing Function for query posting since for a new-line "shift + enter" is needed when browser is full screen

def enter_query(query, prompt_box,driver):
    ip_str = ""
    for ch in query:
        if ch!='\n':
            ip_str = ip_str + ch
        else:
            prompt_box.send_keys(ip_str)
            ActionChains(driver)\
                .key_down(Keys.SHIFT)\
                .send_keys(Keys.ENTER)\
                .key_up(Keys.SHIFT)\
                .perform()
            ip_str = ""
    prompt_box.send_keys(ip_str)


In [11]:
# Counter to keep track on number of queries posted
# Used in bringing back the correct response corresponding to the query

gbl_count = 0

In [12]:

# Function to post query and get the response element back

def make_query(driver, query, gbl_count):

    i = gbl_count
    
    # Load Buttons
    prompt_box = driver.find_element(by=By.CSS_SELECTOR, value="#prompt-textarea")

    # Make Query        
    enter_query(query, prompt_box, driver)
    
    # Wait prompt end
    prompt_box.send_keys("\n")
    
    # Wait for response to finish
    wait_response(driver)
    
    # Return Response Element
    response_element = driver.find_element(by=By.XPATH, value=f"/html/body/div[1]/div[1]/div[2]/main/div[1]/div[1]/div/div/div/div[{2*gbl_count}]/div/div/div[2]/div/div[1]/div/div")
    return response_element

In [13]:
# Main Function

for i in range(5):
        
    # Store Response 
    gbl_count = gbl_count + 1
    response_element = make_query(driver, queries[i], gbl_count)
    
    if response_element.text == "Something went wrong. If this issue persists please contact us through our help center at help.openai.com.":
        print(f"Error Occurred at {i+1}th Query")
        break
    
    html_resp = response_element.get_attribute('innerHTML')
    print("Response to Query " + str(i+1) + " is:")
    print(html_resp)
    f = open(f"response_{i+1}.txt", "w")
    f.write(html_resp)
    f.close()

Response to Query 1 is:
<p>Here's the list of emotion labels for the provided sentences:</p><p>["surprise", "joy", "anger", "contempt", "sadness", "contempt", "contempt", "contempt", "contempt", "neutral", "neutral", "neutral"]</p><p>Please note that the emotion labels have been assigned based on the content of the sentences, as per your instructions.</p>
Response to Query 2 is:
<p>Here's the list of emotion labels for the provided sentences:</p><p>["neutral", "surprise", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral"]</p><p>Please note that the emotion labe